In [1]:
import torch
import torch
import sys
import os
# sys.path.insert(0, os.path.abspath('../src/pretrained/MedViT'))
sys.path.insert(0, os.path.abspath('../'))


from src.pretrained.MedViT import MedViT 
# from MedViT import MedViT_small as tiny
                                       
model_pth = '../medvit_mnist__base_wt75.pt'


c:\Users\rrahman3\AppData\Local\Programs\Python\Python312\Lib\site-packages\timm\models\layers\__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
c:\Users\rrahman3\AppData\Local\Programs\Python\Python312\Lib\site-packages\timm\models\registry.py:4: FutureWarning: Importing from timm.models.registry is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)


In [2]:
import os
print(os.getcwd())

c:\Users\rrahman3\PhDDissertation\EUFairGAN\notebooks


In [3]:
model = MedViT.MedViT_base()
model.proj_head[0] = torch.nn.Linear(in_features=1024, out_features=14, bias=True)

initialize_weights...


In [4]:
model.load_model(model_pth)

c:\Users\rrahman3\PhDDissertation\EUFairGAN\src\models\base_model.py:18: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.load_state_dict(torch.load(path, map_location=tor

Model loaded from ../medvit_mnist__base_wt75.pt


In [5]:
from src.dataloader.medical_dataset import NIHChestXrayDataset
from torch.utils.data import DataLoader
dataset = NIHChestXrayDataset(
            metadata_file="../data/nihcc_chest_xray/nihcc_chest_xray_testing_samples.csv",
            image_dir="../data/nihcc_chest_xray/xray_images/",
        )
dataloader = DataLoader(
    dataset,
    batch_size=16,
    shuffle=True,
    num_workers=1
)

Index(['Unnamed: 0', 'id', 'Atelectasis', 'Cardiomegaly', 'Consolidation',
       'Edema', 'Effusion', 'Emphysema', 'Fibrosis', 'Hernia', 'Infiltration',
       'Mass', 'Nodule', 'Pleural Thickening', 'Pneumonia', 'Pneumothorax',
       'Pneumoperitoneum', 'Pneumomediastinum', 'Subcutaneous Emphysema',
       'Tortuous Aorta', 'Calcification of the Aorta', 'No Finding', 'subj_id',
       'Image Index', 'Finding Labels', 'Follow-up #', 'Patient ID',
       'Patient Age', 'Patient Gender', 'View Position', 'OriginalImage[Width',
       'Height]', 'OriginalImagePixelSpacing[x', 'y]'],
      dtype='object')
21081


In [6]:

with torch.no_grad():
    for image, _, labels in dataloader:
        predictions = model(image)
        print(labels.shape)
        break

torch.Size([16, 14])


In [7]:
pred_sigmoid = torch.sigmoid(predictions)
y_pred_binary = (pred_sigmoid > 0.5).float()

In [12]:
idx = 8
print(pred_sigmoid[idx])
print(y_pred_binary[idx])
print(labels[idx])

tensor([2.0812e-03, 3.6202e-04, 9.0450e-01, 6.1444e-03, 8.4785e-04, 1.6014e-03,
        1.7948e-03, 4.2867e-05, 4.7281e-03, 9.7188e-05, 5.6178e-05, 2.0561e-04,
        2.3714e-04, 4.3134e-02], grad_fn=<SelectBackward0>)
tensor([0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([0., 0., 0., 1., 0., 1., 0., 1., 0., 0., 0., 0., 1., 0.])


In [24]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score, confusion_matrix

In [18]:
pred_binary = (pred_sigmoid.detach().cpu().numpy() > 0.5).astype(int)
labels_binary = labels.detach().cpu().numpy()

accuracy = accuracy_score(labels_binary, pred_binary)
accuracy

0.5625

In [26]:
from sklearn.metrics import confusion_matrix

# Assuming labels_binary and pred_binary are multilabel arrays
for i in range(labels_binary.shape[1]):
    y_true_label = labels_binary[:, i]
    y_pred_label = pred_binary[:, i]
    
    cm = confusion_matrix(y_true_label, y_pred_label)
    print(f"Confusion matrix for label {i}:")
    print(cm)

Confusion matrix for label 0:
[[15  1]
 [ 0  0]]
Confusion matrix for label 1:
[[16]]
Confusion matrix for label 2:
[[14  1]
 [ 1  0]]
Confusion matrix for label 3:
[[14  0]
 [ 2  0]]
Confusion matrix for label 4:
[[14  0]
 [ 2  0]]
Confusion matrix for label 5:
[[15  0]
 [ 1  0]]
Confusion matrix for label 6:
[[16]]
Confusion matrix for label 7:
[[15  0]
 [ 1  0]]
Confusion matrix for label 8:
[[16]]
Confusion matrix for label 9:
[[16]]
Confusion matrix for label 10:
[[16]]
Confusion matrix for label 11:
[[15  0]
 [ 1  0]]
Confusion matrix for label 12:
[[14  0]
 [ 2  0]]
Confusion matrix for label 13:
[[16]]


c:\Users\rrahman3\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:409: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(
c:\Users\rrahman3\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:409: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(
c:\Users\rrahman3\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:409: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(
c:\Users\rrahman3\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_c

In [25]:
confusion_matrix(labels_binary, pred_binary)

ValueError: multilabel-indicator is not supported

In [21]:
f1_score(labels_binary, pred_binary, average='micro')

0.0

In [23]:
import numpy as np
if len(np.unique(labels_binary)) == 2:
    auc = roc_auc_score(labels_binary, pred_sigmoid.detach().cpu().numpy(), multi_class='ovo')
    print(f"ROC AUC: {auc}")
else:
    print("Only one class present in y_true. Skipping AUC calculation.")



ValueError: Only one class present in y_true. ROC AUC score is not defined in that case.

In [22]:
roc_auc_score(labels.detach().cpu().numpy(), pred_sigmoid.detach().cpu().numpy(), multi_class='ovo')

ValueError: Only one class present in y_true. ROC AUC score is not defined in that case.